In [ ]:
import sagemaker
import boto3

In [ ]:
sm = sagemaker.Session()
bucket = 'buck9'
path = 'data'
role = sagemaker.get_execution_role()

In [ ]:
s3 = boto3.client('s3')
contents = s3.list_objects(Bucket=bucket, Prefix=path)['Contents']
for content in contents:
    print(content['Key'])

In [31]:
data = ''
files = ['file1','file2','file3','file4','file5']
for file in files:
    text = s3.get_object(Bucket = bucket,Key = 'data/'+str(file)+'.txt')
    text['Body']
    with io.FileIO('sample.txt', 'w') as file2:
        for i in text['Body']:
            file2.write(i)
    f = open("sample.txt", "r")
    data = data + f.read()
data

In [ ]:
import pandas as pd
import io
import os

In [39]:
df = pd.read_csv(io.StringIO(data), sep =".",header = None).T
df.columns = ['sentence']
df

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [ ]:
corpus = []
for i in range(0,30):
  sentence = re.sub('[^a-zA-Z]',' ',str(df['sentence'][i]))
  sentence = sentence.lower() 
  sentence = sentence.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  sentence = [ps.stem(word) for word in sentence if not word in set(all_stopwords)]
  sentence = ' '.join(sentence)
  corpus.append(sentence)
corpus

In [58]:
result = (" ".join(corpus))
f = open("results.txt", "a")
f.write(result)
boto3.resource('s3').Bucket(bucket).Object(os.path.join('results','results.txt')).upload_file('results.txt')
result

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
encoded_words = le.fit_transform(result.split(" "))
encoded_words

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
wcss = []
for i in range(1, 5):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(encoded_words.reshape(-1,1))
    wcss.append(kmeans.inertia_)
plt.plot(range(1, 5), wcss)
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [69]:
kmeans = KMeans(n_clusters = 2, init = 'k-means++', random_state = 42)
y_kmeans = kmeans.fit_predict(encoded_words.reshape(-1,1))

In [ ]:
for i in range(len(encoded_words)):
    print(encoded_words[i]," ==> K = ",y_kmeans[i])